# Week 7, Day 6: Computer Vision Hackathon Challenge

## Challenge Overview
Build an end-to-end computer vision solution using concepts learned throughout Week 7:
- Image Processing
- Object Detection
- Image Segmentation
- Face Analysis
- Image Generation

## Problem: Multi-Task Vision System
Create a system that can perform multiple computer vision tasks:
1. Object Detection and Recognition
2. Scene Segmentation
3. Face Analysis
4. Image Enhancement

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
import torch
import torchvision

## Part 1: Data Preparation

In [ ]:
def load_and_preprocess_data():
    """Load and preprocess dataset"""
    # Load images
    image_paths = [
        'street_scene1.jpg',
        'street_scene2.jpg',
        'indoor_scene1.jpg',
        'indoor_scene2.jpg'
    ]
    
    images = []
    for path in image_paths:
        # Read image
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Resize
        img = cv2.resize(img, (224, 224))
        
        # Normalize
        img = img.astype(np.float32) / 255.0
        
        images.append(img)
    
    return np.array(images)

# Load data
images = load_and_preprocess_data()

# Display sample images
plt.figure(figsize=(15, 5))
for i in range(min(4, len(images))):
    plt.subplot(1, 4, i+1)
    plt.imshow(images[i])
    plt.title(f'Image {i+1}')
    plt.axis('off')
plt.show()

## Task 1: Object Detection

In [ ]:
def implement_object_detection():
    """Implement object detection system"""
    # Load pre-trained model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    
    def detect_objects(image):
        # Convert to torch tensor
        img_tensor = torch.from_numpy(image.transpose(2, 0, 1)).float()
        img_tensor = img_tensor.unsqueeze(0)
        
        # Get predictions
        with torch.no_grad():
            predictions = model(img_tensor)
        
        return predictions[0]
    
    # Process sample image
    results = detect_objects(images[0])
    
    # Visualize results
    plt.figure(figsize=(10, 10))
    plt.imshow(images[0])
    
    # Draw boxes
    for box, score, label in zip(results['boxes'], results['scores'], results['labels']):
        if score > 0.5:  # Confidence threshold
            x1, y1, x2, y2 = box.numpy()
            plt.gca().add_patch(plt.Rectangle(
                (x1, y1), x2-x1, y2-y1,
                fill=False, color='red', linewidth=2
            ))
    
    plt.title('Object Detection Results')
    plt.axis('off')
    plt.show()

implement_object_detection()

## Task 2: Scene Segmentation

In [ ]:
def implement_segmentation():
    """Implement semantic segmentation"""
    # Load pre-trained model
    model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True)
    model.eval()
    
    def segment_image(image):
        # Preprocess
        img_tensor = torch.from_numpy(image.transpose(2, 0, 1)).float()
        img_tensor = img_tensor.unsqueeze(0)
        
        # Get predictions
        with torch.no_grad():
            output = model(img_tensor)['out'][0]
        
        return output.argmax(0).numpy()
    
    # Process sample image
    segmentation_map = segment_image(images[0])
    
    # Visualize results
    plt.figure(figsize=(15, 5))
    
    plt.subplot(121)
    plt.imshow(images[0])
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(122)
    plt.imshow(segmentation_map)
    plt.title('Segmentation Map')
    plt.axis('off')
    
    plt.show()

implement_segmentation()

## Task 3: Face Analysis

In [ ]:
def implement_face_analysis():
    """Implement face analysis system"""
    # Load face cascade
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
    )
    
    def analyze_face(image):
        # Convert to grayscale
        gray = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2GRAY)
        
        # Detect faces
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        return faces
    
    # Process sample image
    faces = analyze_face(images[0])
    
    # Visualize results
    plt.figure(figsize=(10, 10))
    plt.imshow(images[0])
    
    # Draw rectangles around faces
    for (x, y, w, h) in faces:
        plt.gca().add_patch(plt.Rectangle(
            (x, y), w, h,
            fill=False, color='green', linewidth=2
        ))
    
    plt.title('Face Detection Results')
    plt.axis('off')
    plt.show()

implement_face_analysis()

## Task 4: Image Enhancement

In [ ]:
def implement_enhancement():
    """Implement image enhancement"""
    def enhance_image(image):
        # Convert to LAB color space
        lab = cv2.cvtColor((image * 255).astype(np.uint8), cv2.COLOR_RGB2LAB)
        
        # Split channels
        l, a, b = cv2.split(lab)
        
        # Apply CLAHE to L channel
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(l)
        
        # Merge channels
        limg = cv2.merge((cl,a,b))
        
        # Convert back to RGB
        enhanced = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
        
        return enhanced / 255.0
    
    # Process sample image
    enhanced_image = enhance_image(images[0])
    
    # Visualize results
    plt.figure(figsize=(15, 5))
    
    plt.subplot(121)
    plt.imshow(images[0])
    plt.title('Original Image')
    plt.axis('off')
    
    plt.subplot(122)
    plt.imshow(enhanced_image)
    plt.title('Enhanced Image')
    plt.axis('off')
    
    plt.show()

implement_enhancement()

## Evaluation Criteria

Your solution will be evaluated based on:

1. Object Detection (25%)
   - Detection accuracy
   - Processing speed
   - Implementation quality

2. Segmentation (25%)
   - Segmentation accuracy
   - Edge precision
   - Class coverage

3. Face Analysis (25%)
   - Detection accuracy
   - Feature extraction
   - Analysis depth

4. Enhancement (25%)
   - Image quality
   - Processing efficiency
   - Result naturalness

## Submission Guidelines
1. Complete all tasks in this notebook
2. Document your approach and decisions
3. Include visualizations and analysis
4. Provide suggestions for improvement